In [1]:
from pyforest import *
import pandas as pd
from pathlib import Path
from functional import seq
from tqdm import tqdm

In [2]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/home/dario/workspace/liyuan_paper/zdclink2-14952da40bc3.json


## 翻译

In [3]:
#导入需要的包
import pandas as pd
import numpy as np
import requests
import json
def youdao_translate(word):
    # 有道词典 api
    url = 'http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=null'
    # 传输的参数，其中 i 为需要翻译的内容
    key = {
        'type': "AUTO",
        'i': word,
        "doctype": "json",
        "version": "2.1",
        "keyfrom": "fanyi.web",
        "ue": "UTF-8",
        "action": "FY_BY_CLICKBUTTON",
        "typoResult": "true"
    }
    # key 这个字典为发送给有道词典服务器的内容
    response = requests.post(url, data=key)
    # 判断服务器是否相应成功
    if response.status_code == 200:
        # 然后相应的结果
        return response.text
    else:
        print("有道词典调用失败")
        # 相应失败就返回空
        return None
 
 
def trans_text(word):
    list_trans = youdao_translate(word)
    result = json.loads(list_trans)
    result = result['translateResult'][0][0]['tgt']
    return result

#调用
trans_text('Hello')
#结果《The 21st century》

'你好'

In [4]:
# !export GOOGLE_APPLICATION_CREDENTIALS=/home/dario/workspace/liyuan_paper/zdclink2-14952da40bc3.json

In [5]:
# !python3.8 -m pip install -q googletrans==3.1.0a0

# from googletrans import Translator
# translator = Translator()
# def google_trans(text):
#     return translator.translate(text, dest="zh-cn").text

# google_trans("Hello")

## 数据

In [6]:
root_dir = Path("/home/dario")
data_file = root_dir / "workspace/liyuan_paper/data/GSE42057_series_matrix.txt"

In [7]:
df = pd.read_csv(data_file, delimiter="\t", skiprows=78, index_col=0)

In [8]:
df.dropna(inplace=True)

In [9]:
tmp = !head -17 data/GPL570-55999.txt

### 映射

In [10]:
map_df = pd.read_csv("data/GPL570-55999.txt", sep="\t", skiprows=16, index_col=0)

/home/dario/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [44]:
map_df["Process"] = (
    seq(map_df["Gene Ontology Biological Process"])
     .map(lambda x: x if isinstance(x, str) else "None//None//None")
     .map(lambda x: x.split("///"))
     .map(lambda x: [y.split("//")[1].strip() for y in x])
     .map(lambda x: ','.join(x))
    #  .map(lambda x: [y for y in x if not (y.startswith("inferred from") or y.startswith("traceable author statement"))])
).to_list()

In [45]:
map_df = map_df[["Gene Title", "Process"]]

In [48]:
map_df.fillna("None", inplace=True)

/home/dario/.local/lib/python3.8/site-packages/pandas/core/frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [50]:
map_df.loc["117_at", "Gene Title"]
map_df.loc["117_at", "Process"]

'cell morphogenesis,ATP catabolic process,response to stress,response to unfolded protein,cellular response to heat,protein refolding,cellular heat acclimation'

In [51]:
token_2_title = dict(zip(map_df[["Gene Title"]].index, map_df["Gene Title"].values))

In [13]:
# import time
# with open("data/GSE42057_token_2_title.txt", "w") as f:
#     for token in tqdm(df.index):
#         try:
#             f.write(f"{token}\t{google_trans(token_2_title.get(token))}\n")
#         except Exception:
#             pass
#         time.sleep(2)

## 计算

In [52]:
import causal_discovery
causal_discovery.__version__

'1.0.5'

In [53]:
from causal_discovery.parameter.log import config_log
config_log(
    "causal_discovery",
    "local_ng_cd",
    log_root="/home/dario/workspace/liyuan_paper/log",
    print_terminal=False,
    enable_monitor=True,
)

In [54]:
from causal_discovery.algorithm.local_ng_cd.local_ng_cd import local_ng_cd
from causal_discovery.data_prepare.matrix_data import get_matrix_data
from causal_discovery.parameter.algo import DataType, LocalNgCdParam
from causal_discovery.parameter.env import select_xp
from causal_discovery.parameter.error import DataTypeError

xp = select_xp()

In [55]:
def get_causal_result(target):
    matrix_data = get_matrix_data(
        target,
        corr_filter=True,
        need_norm=False,
        ret_df=df.T,
    ).T

    # 暴露出来允许修改的参数
    param = LocalNgCdParam()
    param.candidate_two_step = False
    param.target_index = list(matrix_data.columns).index(target)
    # 变量token -> name
    index_map = dict(enumerate(matrix_data.columns))
    # 调用主函数计算因果
    edges_trust, synthesize_effect = local_ng_cd(
        xp.asarray(matrix_data.T.to_numpy()), param, synthesize=True
    )
    edges_trust = [
        (index_map.get(int(causal), "N"), index_map.get(int(result), "N"), weight)
        for causal, result, weight in edges_trust
    ]
    return edges_trust

In [19]:
target = "1294_at"
trans_text(token_2_title[target])

'microRNA 5193 / / / ubiquitin-like修改器激活酶7'

In [20]:
edges_trust = get_causal_result(target)
pd.DataFrame([(trans_text(token_2_title[x]), trans_text(token_2_title[y]), z) for x, y, z in edges_trust])

,0,1,2
0,排序nexin 6,microRNA 5193 / / / ubiquitin-like修改器激活酶7,10.222674
1,microRNA 5193 / / / ubiquitin-like修改器激活酶7,translin,-13.021033


In [57]:
trans_text(x_process)

'蛋白质磷酸化酶联受体蛋白信号通路,磷酸化,peptidyl-tyrosine磷酸化,cytokine-mediated信号通路、细胞内信号转导,interleukin-2-mediated信号通路,应对抗生素,蛋白质自身磷酸化,interferon-gamma-mediated信号通路,调节interferon-gamma-mediated信号通路,I型干扰素信号通路,调节I型interferon-mediated信号通路'

In [63]:
import sys

In [69]:
errors = []
with open("data/result.csv", "w") as f:
    f.write("target\ttoken_x\ttoken_y\ttitle_x\ttitle_y\tprocess_x\tprocess_y\tweight\n")
    for target in tqdm(df.index):
        try:
            target_name = trans_text(token_2_title[target])
            edges_trust = get_causal_result(target)
            for x, y, z in edges_trust:
                x_name = trans_text(token_2_title[x])
                y_name = trans_text(token_2_title[y])
                x_process = trans_text(map_df.loc[x, "Process"])
                y_process = trans_text(map_df.loc[y, "Process"])
                f.write(f"{target}\t{x}\t{y}\t{x_name}\t{y_name}\t{x_process}\t{y_process}\t{z}\n")
        except Exception as e:
            print(sys.exc_info())
            errors.append(target)

  0%|          | 0/12531 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [62]:
errors

['117_at',
 '1294_at',
 '1552256_a_at',
 '1552257_a_at',
 '1552263_at',
 '1552264_a_at',
 '1552275_s_at',
 '1552277_a_at',
 '1552287_s_at',
 '1552291_at',
 '1552309_a_at',
 '1552310_at',
 '1552315_at',
 '1552316_a_at',
 '1552318_at',
 '1552321_a_at',
 '1552323_s_at',
 '1552329_at',
 '1552347_at',
 '1552370_at']